In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from keras.utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Dense
from keras.models import Sequential
from keras.metrics import Precision, Recall 
from keras.callbacks import ModelCheckpoint

In [3]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report

In [4]:
img_h = 224
img_w = 224
batch_size = 12
epoch = 10
train_path = "DogsDatasets/Train"
test_path = "DogsDatasets/Test"

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_dataset = train_datagen.flow_from_directory(train_path,
                                                  target_size=(img_h, img_w),
                                                  class_mode='categorical',
                                                  color_mode='rgb',
                                                  shuffle=False)
test_dataset = test_datagen.flow_from_directory(test_path,
                                                  target_size=(img_h, img_w),
                                                  class_mode='categorical',
                                                  color_mode='rgb',
                                                  shuffle=False)

Found 17700 images belonging to 120 classes.
Found 4426 images belonging to 120 classes.


In [7]:
checkpoint = ModelCheckpoint("Dogs-Model-1.h5",
                             monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',
                             save_freq='epoch')

In [9]:
model = Sequential([
    Conv2D(filters=64, kernel_size=7, strides=2, padding='same', activation='relu', input_shape=train_dataset.image_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),
    
    Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    GlobalAveragePooling2D(),

    Dense(1000, activation='relu'),
    Dense(120, activation='softmax'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      9472      
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                        

In [ ]:
METRICS = [
        'accuracy',
        Precision(name='precision'),
        Recall(name='recall')
    ]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [ ]:
model_history = model.fit(train_dataset,
                          batch_size=batch_size,
                          validation_data=test_dataset,
                          epochs=epoch,
                          callbacks=[checkpoint])

In [ ]:
model.save("Dogs-Model-1.h5")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))
ax = ax.ravel()

metrics = ['accuracy', 'precision', 'recall', 'loss']
for i, met in enumerate(metrics):
    row = i // 2
    col = i % 2
    ax[row,col].plot(model_history.history[met])
    ax[row,col].plot(model_history.history['val_' + met])
    ax[row,col].set_title('Model {}'.format(met))
    ax[row,col].set_xlabel('epochs')
    ax[row,col].set_ylabel(met)
    ax[row,col].legend(['train', 'val'])

plt.tight_layout()
plt.show()

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
labels = []

for key, value in train_dataset.class_indices.items():
   labels.append(key)

In [ ]:
def evaluate(actual, predictions):
  pre = []
  for i in predictions:
    pre.append(np.argmax(i))

  accuracy = (pre == actual).sum() / actual.shape[0]
  print(f'Accuracy: {accuracy}')

  precision, recall, f1_score, _ = precision_recall_fscore_support(actual, pre, average='macro')
  print(f'Precision: {precision}')
  print(f'Recall: {recall}')
  print(f'F1_score: {f1_score}')

  fig, ax = plt.subplots(figsize=(8,8))
  conf_mat = confusion_matrix(actual, pre)
  sns.heatmap(conf_mat, annot=True, fmt='.0f', xticklabels=labels, yticklabels=labels).set_title('Confusion Matrix Heat map')
  plt.show()

In [ ]:
evaluate(test_dataset.classes,predictions)

In [ ]:
model.evaluate(test_dataset)